In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<div class="alert alert-success">
  Según distintos benchmarks (papers, kaggle.com) los **algoritmos de uso general** que tienen más veces la mejor performance son: **Gradient Boosted Trees, Random Forest y SVM**.
</div>

<img src="./img/benchmark_algs.png">

## Decision Trees

In [2]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=2)

In [3]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('./data/transformed/peliculas_opening.csv')
y = X['worldwide_gross']

X.drop(columns=['worldwide_gross', 'movie_title'], inplace=True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)
model.fit(X_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [5]:
import pydotplus
from io import StringIO 

from sklearn.tree import export_graphviz
out = StringIO()

treedot = export_graphviz(model,
                         out_file=out,
                         feature_names=X.columns)

graph = pydotplus.graph_from_dot_data(out.getvalue())
graph.write_png('./img/arbol.png')

True

<img src="./img/arbol.png" width="600">

Virtudes de los arboles de decision:

- Metodo poderoso y probado
- Interpretable
- No necesita escalar los datos (clasificación), y menos preprocesamiento de variables

Sin embargo en la practica existen modelos que obtienen mejor rendimiento. Como mejorar el modelo de arboles de decisión?

## Ensembles

**Concepto General**

Random Forest y Gradient Boosted Trees, forman parte de una familia de algoritmos que se denominan ensembles.

$$ Ensemble = Submodelos \rightarrow Entrenamiento \rightarrow Predicciones_{Intermedias} \rightarrow Voto \rightarrow Prediccion_{final}$$ 

** Cómo funciona el algoritmo Random Forest?**

Vamos a generar cientos de modelos de arboles de decisión que serán entrenados sobre **conjuntos de datos bootstrapeados** del conjunto de datos original y donde para cada etapa de separación el **conjunto de features elegibles** sera un subconjunto aleatorio del conjunto original de features.

<img src="./img/rf_tree.png" width="600">

Cada uno de los arboles entrenados luego podrá votar por su predicción y promediaremos estos votos.

<img src="./img/random_forest.png" width="600">

**Ensembles del pobre ("Poor man's ensembles")**

- Entrenar diversos modelos a mano
- Promediar el resultado
- Owen Zhang, número 1 de Kaggle.com durante un largo tiempo, ocupaba esta estrategia promediando diversos modelos XGBoost.
- ``from sklearn.ensemble import VotingClassifier`` sirve por ejemplo para hacer un ensemble manual de clasificación

En general los ensembles del pobre funcionan ya que cada uno de los modelos que votarán en conjunto son bastante fuertes.

**Porqué RF es poderoso?**

<div class="alert alert-success">
  **Leo Breiman** creador del Random Forest demostró que un ensemble podía tener buen poder de generalización sí:
  <ol>
    <li>Los submodelos tienen buen poder de predicción</li>
    <li>Los submodelos están descorrelacionados</li>
    </ol>
</div>

Así el algoritmo de Random Forest compromete un poco de poder de predicción de cada uno de los decision trees que arma, pero la forma aleatoria de generarlos hace que esten **fuertemente descorrelacionados**.

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

forest = RandomForestRegressor(200)
results = cross_validate(forest,X,y,cv=5,scoring='r2', return_train_score=True)
results

{'fit_time': array([2.92849016, 3.08679914, 3.07547259, 3.03957987, 3.16622925]),
 'score_time': array([0.09733891, 0.04001951, 0.06779099, 0.04801083, 0.05201268]),
 'test_score': array([0.45687818, 0.6527866 , 0.5605595 , 0.38749289, 0.45367076]),
 'train_score': array([0.96093835, 0.96782554, 0.96370828, 0.96399208, 0.96690559])}

In [7]:
test_scores = results['test_score']
train_scores = results['train_score']

print('{:.2%}'.format( np.mean(train_scores) ) )
print('{:.2%}'.format( np.mean(test_scores) ) )

96.47%
50.23%


Mejor resultado que Lasso! Ya no tenemos Bias y tenemos un mejor score r2. Sin embargo tenemos una diferencia importante entre score de entrenamiento y de test (overfit).

## Gradient Boosted Trees

In [8]:
from sklearn.ensemble import GradientBoostingRegressor

ensemble = GradientBoostingRegressor()
results = cross_validate(ensemble,X,y,cv=5,scoring='r2', return_train_score=True)
results

{'fit_time': array([0.61750984, 0.72979712, 0.56515074, 0.67369843, 0.73229003]),
 'score_time': array([0.00800443, 0.00796962, 0.00400162, 0.00400019, 0.00400162]),
 'test_score': array([0.41157558, 0.64459919, 0.59878371, 0.48559503, 0.40603166]),
 'train_score': array([0.85453936, 0.93117267, 0.92752691, 0.92546652, 0.92410658])}

In [9]:
test_scores = results['test_score']
train_scores = results['train_score']

print('{:.2%}'.format( np.mean(train_scores) ) )
print('{:.2%}'.format( np.mean(test_scores) ) )

91.26%
50.93%


Cómo optimizamos los parametros de este último modelo?

## Optimización de hiperparametros

- Fijar un learning rate alto
- Fijar parametros de los arboles
- Fijados estos parametros, elegir el mejor numero de estimadores que conforman el ensemble
- (Tarea) Con el learning rate dado y el numero de estimadores óptimo, optimizar los parametros de los arboles

**Grid Search**

Por ahora dijimos que:
    
- train_test_split servia para evaluaciones rapidas, testeos y prototipaje
- cross_validate es un método más robusto para poder estimar el rendimiento de tu algoritmo

Sin embargo una vez que hemos finalizado nuestra etapa de prototipaje y ya queremos establecer un modelo definitivo deberiamos seguir el flujo siguiente.

<img src="./img/grid_search_crossval.png" width=700>

In [10]:
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)
param_test1 = {'n_estimators': range(20,501,20)}
display(param_test1) 
display(list(param_test1['n_estimators']))

{'n_estimators': range(20, 501, 20)}

[20,
 40,
 60,
 80,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 440,
 460,
 480,
 500]

In [11]:
estimator = GradientBoostingRegressor(learning_rate=0.1,
                                     min_samples_split=500,
                                     min_samples_leaf=50,
                                     max_depth=8,
                                     max_features='sqrt',
                                     subsample=0.8,
                                     random_state=10)
estimator

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=8,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=50, min_samples_split=500,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=10, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [12]:
gsearch1 = GridSearchCV(estimator,
                       param_grid = param_test1,
                       scoring='r2',
                       cv=5,
                       return_train_score=True)
gsearch1

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=8,
                                                 max_features='sqrt',
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=50,
                                                 min_samples_split=500,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change=None,
            

In [13]:
gsearch1.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=8,
                                                 max_features='sqrt',
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=50,
                                                 min_samples_split=500,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change=None,
            

In [14]:
gsearch1.best_params_, '{:.2%}'.format(gsearch1.best_score_)

({'n_estimators': 160}, '75.16%')

In [15]:
final_results = cross_validate(gsearch1.best_estimator_,X_train,y_train, return_train_score=True)
final_results

{'fit_time': array([0.70727515, 0.48398018, 0.43707776, 0.57841849, 0.55507946]),
 'score_time': array([0.0040009 , 0.00400186, 0.00399709, 0.01200294, 0.00799966]),
 'test_score': array([0.66304799, 0.87141642, 0.6543357 , 0.78222026, 0.78683568]),
 'train_score': array([0.83258316, 0.79135576, 0.81938793, 0.80362876, 0.80880346])}

In [16]:
test_scores = final_results['test_score']
train_scores = results['train_score']

print('{:.2%}'.format( np.mean(train_scores) ) )
print('{:.2%}'.format( np.mean(test_scores) ) )

91.26%
75.16%


In [17]:
estimator = GradientBoostingRegressor(learning_rate=0.1,
                                     min_samples_split=500,
                                     min_samples_leaf=50,
                                     max_depth=8,
                                     max_features='sqrt',
                                     subsample=0.8,
                                     random_state=10,
                                     n_estimators=160)
                                     
estimator.fit(X_train,y_train)

score = estimator.score(X_test,y_test)
'{:.2%}'.format(score)

'78.25%'

## Reflexiones de cierre

** Recursos **

- Reddit /machinelearning y /learnmachinelearning
- Analytics Vidhya y KD Nuggets
- Kaggle.com y "There is no Free Hunch" Blog
- Arxiv, papers
- Libros: "Pattern Recognition and Machine Learning" C.Bishop y "Elements of Statistical Learning".

** Próximos pasos **

- Matemáticas
- Praxis: Feature Engineering, Model Selection y Tuning
- Deep Learning para NLP y Computer Vision
- Machine Learning Bayesiano